In [1]:
# 6. 실행 파일 1 (최적화 비중 뽑아내는 것 까지)

import pandas as pd
import numpy as np
import os
import config
from data_manager import DataManager
from portfolio_optimizer import PortfolioOptimizer

# 1. 데이터 가져오기
dm = DataManager()
stock_data = dm.get_data(config.STOCKS, config.START_DATE, config.END_DATE, save_path=config.RAW_DATA_PATH)

# 2. 최적화 실행
if not stock_data.empty:
    results = {}
    final_tickers = stock_data.columns.tolist()
    num_assets = len(final_tickers)
    daily_returns = dm.calculate_returns(stock_data, 'daily')
    optimizer = PortfolioOptimizer(daily_returns)

    for name, params in config.STRATEGIES.items():
        if not params.get('enabled', False):
            continue
            
        print(f"\n--- '{name}' 모델 분석 시작 ---")
        
        if params.get('is_optimizer') is False:
            weights = optimizer.get_risk_parity_weights(daily_returns)
        else:
            init_guess = np.array([1/num_assets] * num_assets)
            bounds = tuple((0, params.get('bounds_cap', 1.0)) for _ in range(num_assets))
            constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
            target_return = params.get('target_return')
            
            if target_return is not None:
                constraints.append({'type': 'ineq', 'fun': lambda w: optimizer.calculate_metrics(w)['return'] - target_return})

            weights = optimizer.run_optimization(params['objective'], constraints=constraints, bounds=bounds)
        
        results[name] = {'weights': weights}
        print(f"최적 가중치: " + ", ".join([f"{s}: {w*100:.2f}%" for s, w in zip(final_tickers, weights)]))

    # 3. 결과 저장
    if results:
        os.makedirs(config.RESULTS_DIR, exist_ok=True)
        with pd.ExcelWriter(config.OPTIMAL_WEIGHTS_SAVE_PATH, engine='openpyxl') as writer:
            for model_name, data in results.items():
                weights_df = pd.DataFrame({'Ticker': final_tickers, 'Optimal_Weight': data['weights']})
                weights_df.to_excel(writer, sheet_name=f'{model_name}_weights', index=False)
        print(f"\n모든 모델의 최적 가중치가 저장되었습니다: {config.OPTIMAL_WEIGHTS_SAVE_PATH}")


'['SPY', 'QQQ', 'EFA', 'EEM', 'SSO', 'TQQQ', 'TLT', 'IEF', 'SHY', 'GLD', 'DBC', 'BTC-USD', 'ETH-USD', 'XRP-USD']' 종목 데이터 다운로드 시도 중...


[*********************100%***********************]  14 of 14 completed


수정 종가 데이터가 로컬에 저장되었습니다: /Users/012oov/Documents/Quant/MPT_Back/Data/Raw data/downloaded_stock_prices.csv

--- 'max_calmar' 모델 분석 시작 ---
최적 가중치: SPY: 7.05%, QQQ: 0.00%, EFA: 0.00%, EEM: 0.00%, SSO: 0.00%, TQQQ: 0.00%, TLT: 0.00%, IEF: 0.00%, SHY: 71.43%, GLD: 8.30%, DBC: 8.26%, BTC-USD: 3.46%, ETH-USD: 0.00%, XRP-USD: 1.50%

--- 'risk_parity' 모델 분석 시작 ---
최적 가중치: SPY: 5.05%, QQQ: 3.78%, EFA: 5.38%, EEM: 4.78%, SSO: 2.56%, TQQQ: 1.27%, TLT: 5.47%, IEF: 11.54%, SHY: 46.43%, GLD: 5.74%, DBC: 4.67%, BTC-USD: 1.46%, ETH-USD: 1.09%, XRP-USD: 0.79%

--- 'min_variance' 모델 분석 시작 ---
최적 가중치: SPY: 0.03%, QQQ: 0.00%, EFA: 0.00%, EEM: 0.00%, SSO: 0.00%, TQQQ: 0.00%, TLT: 0.00%, IEF: 0.00%, SHY: 98.80%, GLD: 0.00%, DBC: 1.16%, BTC-USD: 0.00%, ETH-USD: 0.00%, XRP-USD: 0.01%

--- 'daily_30_cap' 모델 분석 시작 ---
최적 가중치: SPY: 17.01%, QQQ: 0.00%, EFA: 0.00%, EEM: 0.00%, SSO: 0.00%, TQQQ: 0.00%, TLT: 0.00%, IEF: 0.00%, SHY: 30.00%, GLD: 25.58%, DBC: 16.35%, BTC-USD: 7.78%, ETH-USD: 0.00%, XRP-USD: 3.28%

--- '